In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

In [28]:
import os
import re
import random
import pandas as pd
import json
import sklearn
import spacy
import numpy as np
import torch
import torch.nn.functional as F
from itertools import islice
from tqdm import tqdm
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
sp = spacy.load("en_core_web_sm")

In [3]:
filepath = "../hein-bound/speeches_111.txt"
with open(filepath, errors="ignore") as f:
    speech = f.readlines()
    speech = [s.replace("\n", "").replace(" | ", " ").split("|") for s in speech]
df = pd.DataFrame(speech[1:], columns = speech[0]) 

In [4]:
df.head(10)

,speech_id,speech
0,1110000001,The Representativeselect and their guests will...
1,1110000002,As directed by law. the Clerk of the House has...
2,1110000003,The quor closes that 428 Represer have respond...
3,1110000004,Credentials. regular in form. have been receiv...
4,1110000005,The Clerk is in receipt of a letter of resigna...
5,1110000006,Pursuant to law and precedent. the next order ...
6,1110000007,Our democracy renews itself every 2 years as M...
7,1110000008,The Clerk now recognizes the gentleman from In...
8,1110000009,Madam Clerk. as chairman of the Republican Con...
9,1110000010,The names of the Honorable NANCY PELOSI. a Rep...


In [5]:
corpus = df.speech

In [6]:
corpus

0         The Representativeselect and their guests will...
1         As directed by law. the Clerk of the House has...
2         The quor closes that 428 Represer have respond...
3         Credentials. regular in form. have been receiv...
4         The Clerk is in receipt of a letter of resigna...
                                ...                        
179263    Madam Speaker. on rollcall Nos. 662 and 661. I...
179264    Madam Speaker. as I leave Congress as the peop...
179265    Madam Speaker. on rolicall No. 658. I was unav...
179266    Madam Speaker. on rollcall No. 658 my flight w...
179267    Madam Speaker. I wold like to submit the follo...
Name: speech, Length: 179268, dtype: object

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=5, max_df=.5, ngram_range=(1,3))
vectorizer.fit(corpus)

In [8]:
corpus.iloc[19]

'According to precedent. the Chair will swear in the Memberselect en masse. The Memberselect and Delegateselect and the Resident Commissionerelect rose. and the Speaker administered the oath of office to them as follows: Do you solemnly swear or affirm that you will support and defend the Constitution of the United States against all enemies. foreign and domestic. that you will bear true faith and allegiance to the same. that you take this obligation freely. without any mental reservation or purpose of evasion. and that you will well and faithfully discharge the duties of the office on which you are about to enter. so help you God.'

In [9]:
questions = set([s for s in corpus if s.find('?') != -1])

In [10]:
sorted(questions, key=lambda x: len(x), reverse=False)

['$1?',
 'What?',
 'Extra?',
 'Seven?',
 'Sorry?',
 'Pardon?',
 'Totally?',
 'And then?',
 'Im sorry?',
 'How many?',
 'How much?',
 'Hows that?',
 'The value?',
 'Pardon me?',
 'What page?',
 'Excuse me?',
 'For snakes?',
 'I am sorry?',
 'Will yield?',
 'Was it you?',
 'Ten minutes?',
 'For my side?',
 'How about 1?',
 'Your health?',
 'Senator KYL?',
 'Has to what?',
 'On your time?',
 'There is not?',
 'He said what?',
 'For how long?',
 'Five million?',
 '2.000 people?',
 'What happens?',
 'Breast cancer?',
 'Do we have to?',
 'The answer is?',
 'Mr. President?',
 'How much time?',
 'May I respond?',
 'On whose time?',
 'Then if I may?',
 'Including you?',
 'Is that a "no"?',
 'How many times?',
 'I dont. do you?',
 'Is it paid for?',
 'Is that better?',
 'They come here?',
 'Supporting pie?',
 'Did I say a.m.?',
 'Yield for what?',
 'May I continue?',
 'Why isnt it law?',
 'Would you yield?',
 'Is that unusual?',
 'So its exempted?',
 'Who yields time?',
 'TennCare failed?',
 'Is

In [11]:
speeches = [s for s in corpus if s.find('?') == -1]

In [12]:
len(speeches)

152689

In [13]:
small_speeches = speeches[:10000]

In [14]:
small_speeches[:100]

['The Representativeselect and their guests will please remain standing and join in the Pledge of Allegiance.',
 'As directed by law. the Clerk of the House has prepared the official roll of the Representativeselect. Certificates of election covering 435 seats in the 111th Congress have been received by the Clerk of the House. and the names of those persons whose credentials show that they were regularly elected as Representatives in accordance with the laws of their respective States or of the United States will be called. The Representativeselect will record their presence by electronic device and their names will be recorded in alphabetical order by State. beginning with the State of Alabama. to determine whether a quorum is present. Representativeselect will have a minimum of 15 minutes to record their presence by electronic device. Representativeselect who have not obtained their voting ID cards may do so now in the Speakers lobby.',
 'The quor closes that 428 Represer have respon

### TF.IDF

In [15]:
def lemmatize(phrase):
    return " ".join([word.lemma_ for word in sp(phrase)])

In [16]:
lemmas = [lemmatize(speech) for speech in small_speeches]
df = pd.DataFrame(data={'context': small_speeches, 'lemmas': lemmas})
df.to_feather("context.feather")

In [17]:
#df = pd.read_feather("context.feather")
sm_speeches = df.context
lemmas = df.lemmas

In [18]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=5, max_df=.5, ngram_range=(1,2))
tfidf = vectorizer.fit_transform(lemmas)
len(vectorizer.vocabulary_)

22503

Madam Speaker. let me congratulate you for your reelection as Speaker of the House. It is an honor that you tave served with great distinction and verve. I look forward to more of your continued leadership in the 111th Congress. Mr. Speaker. I rise today in support of H. Res. 5. Adopting the rules for the One Hundred Eleventh Congress. The House Rules Package provides commonsense reforms that will enable Congress to work more efficiently for America. In the 110th Congress. Democrats put forth critical measures to restore integrity and accountability to the House. These reforms were the most sweeping ethics and lobbying reforms since Watergate and has changed the way Congress does business in Washington. 


#### The reforms adopted by the 110th Congress included banning gifts from lobbyists. prohibiting the use of corporate jets. mandating ethics training for all House employees. establishing a new. independent Office of Congressional Ethics. and ensuring transparency for budget earmarks by requiring the full disclosure of earmarks in all bills and conference reports. The Rule Package for the 111th Congress builds upon these reforms to further strengthen the integrity of Congress.

In [19]:
question = "What do Democrats believe about tax increases?"
query = vectorizer.transform([lemmatize(question)])
(query > 0).sum(), vectorizer.inverse_transform(query)

(5, [array(['110th', '110th congress', 'adopt', 'congress', 'reform'],
        dtype='<U30')])

In [26]:
def change_comma(speech):
    """
    Fixes comma issues due to OCR errors
    :param speech: text of the speech
    :return: corrected text
    """
    return re.sub("\.(?=\s[a-z0-9]|\sI[\W\s])", ",", speech)

In [29]:
scores = (tfidf * query.T).toarray()
results = (np.flip(np.argsort(scores, axis=0)))
for i in results[:5, 0]:
    print(change_comma(sm_speeches[i]))
    print()

Madam Speaker, let me congratulate you for your reelection as Speaker of the House. It is an honor that you tave served with great distinction and verve, I look forward to more of your continued leadership in the 111th Congress. Mr. Speaker, I rise today in support of H. Res, 5. Adopting the rules for the One Hundred Eleventh Congress. The House Rules Package provides commonsense reforms that will enable Congress to work more efficiently for America. In the 110th Congress. Democrats put forth critical measures to restore integrity and accountability to the House. These reforms were the most sweeping ethics and lobbying reforms since Watergate and has changed the way Congress does business in Washington. The reforms adopted by the 110th Congress included banning gifts from lobbyists, prohibiting the use of corporate jets, mandating ethics training for all House employees, establishing a new, independent Office of Congressional Ethics, and ensuring transparency for budget earmarks by req

### QA

In [30]:
qa_pipeline = pipeline("question-answering")

In [31]:
THRESHOLD = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:5, 0] ]
contexts = [ (sm_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]

question_df = pd.DataFrame.from_records([ {
    'question': question,
    'context':  ctx
} for (ctx,s) in contexts ])
question_df.to_feather("question_context.feather")

In [32]:
question_df

,question,context
0,What reforms were adopted by the 110th Congress?,Madam Speaker. let me congratulate you for you...
1,What reforms were adopted by the 110th Congress?,would not affect trade or transportation of an...
2,What reforms were adopted by the 110th Congress?,Mr. President. I have sought recognition today...
3,What reforms were adopted by the 110th Congress?,Mr. President. I am pleased to join with the s...
4,What reforms were adopted by the 110th Congress?,Mr. President. pursuant to rule XLIV of the St...


In [33]:
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)
answer_df.head(10)

,score,start,end,answer
3,0.891813,2472,2498,Lobbying and Ethics Reform
0,0.563236,764,793,banning gifts from lobbyists.
4,0.555780,2084,2106,nonpublicland measures
1,0.539387,473,495,bipartisan legislation.
2,0.111105,2464,2487,seconddegree amendment.


In [36]:
def get_short_answers(question, qa_pipeline):
    # vectorize question
    query = vectorizer.transform([lemmatize(question)])
    # compute cosine similarity with set of speeches
    scores = (tfidf * query.T).toarray()
    results = (np.flip(np.argsort(scores, axis=0)))
    
    THRESHOLD = 0.01
    candidate_idxs = [ (i, scores[i]) for i in results[0:5, 0] ]
    contexts = [ (sm_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]
    for i in results[0:5, 0]:
        print(change_comma(sm_speeches[i]))
        print()

    question_df = pd.DataFrame.from_records([ {
        'question': question,
        'context':  ctx
    } for (ctx,s) in contexts ])
    
    preds = qa_pipeline(question_df.to_dict('records'))
    answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)
    return answer_df.head()

In [38]:
get_short_answers("What do you believe about tax increases?", qa_pipeline)

Madam President, what this bill does is probably one of the most important things we need to do in this economic debate, and it is stop the planned tax increases that are going to happen in 2011 for every American. The large score that is being thrown around here assumes we are going to let those taxes go up, but we are not. This is a misrepresentation of the cost of this bill. This bill stops the current tax increases that are planned in 2011, keeps the current tax rate the same. The only change it makes is it lowers the top marginal rate from 35 to 25 percent for businesses, for investors, and for individual Americans. We call it the American option because it leaves money in the hands of the American people and businesses, rather than bringing it to Washington and distributing it our way, I encourage everyone to stop the planned tax increases with the American option.

Madam Speaker, I think that, I would hope, that not only my colleagues but the American people realize that this bi

,score,start,end,answer
0,0.546953,449,485,keeps the current tax rate the same.
4,0.169261,15,41,I believe I have the floor.
1,0.146167,299,350,tax increase that is coming full steam ahead a...
2,0.048881,3242,3296,you will not see your taxes increase by a sing...
3,0.002923,331,366,I believe its going to create jobs.


In [23]:
get_short_answers("How much working women make compared to men in the same position in the United States?",
                  qa_pipeline)

,score,start,end,answer
0,0.857778,21,29,1 minute
4,0.511652,279,287,78 cents
2,0.387330,1989,2011,77 cents on the dollar
3,0.290338,290,312,78 cents on the dollar
1,0.000018,355,404,shoulder the responsibility but dont get the pay.


### Text generation

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

generated = tokenizer.encode("What reforms were adopted by the 110th Congress??")
context = torch.tensor([generated])
past = None

for i in range(100):
    output, past = model(context, past=past)
    token = torch.argmax(output[..., -1, :])

    generated += [token.tolist()]
    context = token.unsqueeze(0)

sequence = tokenizer.decode(generated)

print(sequence)

I0613 00:31:05.810559 140734868266432 tokenization_utils.py:1078] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /Users/alexandrabutoi/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
I0613 00:31:05.811711 140734868266432 tokenization_utils.py:1078] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /Users/alexandrabutoi/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
I0613 00:31:06.453675 140734868266432 configuration_utils.py:265] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /Users/alexandrabutoi/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e0

What reforms were adopted by the 110th Congress??

The first was the "National Defense Authorization Act of 2012," which authorized the Pentagon to spend $1.5 trillion over 10 years to increase the military's budget by $1.5 trillion over 10 years. The bill also included a $1.5 trillion increase in the military's budget for fiscal year 2015.

The second was the "Defense Authorization Act of 2012," which authorized the Pentagon to spend $1.5 trillion over 10 years to increase the military's budget by $


### Doc2vec

In [39]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess

In [40]:
processed = sm_speeches.apply(simple_preprocess)

In [41]:
processed.head()

0    [the, and, their, guests, will, please, remain...
1    [as, directed, by, law, the, clerk, of, the, h...
2    [the, quor, closes, that, represer, have, resp...
3    [credentials, regular, in, form, have, been, r...
4    [the, clerk, is, in, receipt, of, letter, of, ...
Name: context, dtype: object

In [42]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(processed)]
model = Doc2Vec(documents, vector_size=512, window=2, min_count=3, workers=4, epochs=10)

In [51]:
qvector = model.infer_vector(simple_preprocess("What reforms were adopted by the 110th Congress?"))

In [52]:
sims = model.docvecs.most_similar([qvector], topn=2)

In [53]:
sims

[(8639, 0.8414086103439331), (8921, 0.840142011642456)]

In [54]:
docs = [i for i, j in sims]

In [55]:
x = sm_speeches[docs]

In [56]:
x

8639    The majority leader is recognized.
8921      The Senator from South Carolina.
Name: context, dtype: object

In [57]:
for i in x:
    print(change_comma(i))
    print()

The majority leader is recognized.

The Senator from South Carolina.



In [49]:
question_df = pd.DataFrame.from_records([ {
    'question': "What do Democrats believe about tax increases?",
    'context':  ctx
} for ctx in x ])
question_df

,question,context
0,What do Democrats believe about tax increases?,Mr. President. the next vote will be on Monday...
1,What do Democrats believe about tax increases?,Mr. President. I anticipate about 5 minutes.


In [73]:
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)

In [74]:
answer_df

,score,start,end,answer
2,0.290470,154,162,5 oclock.
0,0.235915,771,811,Republicans dont care about health care.
8,0.202437,42,55,some comments
5,0.178406,15,42,I need about 7 or 8 minutes.
7,0.114170,700,764,The House Democrats bill provides enough spend...
3,0.069440,0,14,Mr. President.
9,0.066318,190,216,policies I disagreed with.
6,0.059903,345,390,what opportunity there will be to get a vote.
4,0.025305,88,110,it is instead about en-
1,0.019844,492,548,it Is not a choice between doing nothing on th...


### Universal Sentence Encoder

In [58]:
import tensorflow_hub as hub
from scipy.spatial.distance import cosine
import hnswlib
from sklearn.metrics.pairwise import cosine_similarity

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

INFO:absl:Using /var/folders/dj/27jw3fzj6kgfm7v2h7y9vq7w0000gn/T/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 10.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 20.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 30.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 40.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 50.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 60.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 70.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 80.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 90.00MB
INFO:absl:Downloading https://tfhub.dev/googl

In [59]:
def embed(input):
  return model(input)

In [15]:
small_speeches[:3]

['The Representativeselect and their guests will please remain standing and join in the Pledge of Allegiance.',
 'As directed by law. the Clerk of the House has prepared the official roll of the Representativeselect. Certificates of election covering 435 seats in the 111th Congress have been received by the Clerk of the House. and the names of those persons whose credentials show that they were regularly elected as Representatives in accordance with the laws of their respective States or of the United States will be called. The Representativeselect will record their presence by electronic device and their names will be recorded in alphabetical order by State. beginning with the State of Alabama. to determine whether a quorum is present. Representativeselect will have a minimum of 15 minutes to record their presence by electronic device. Representativeselect who have not obtained their voting ID cards may do so now in the Speakers lobby.',
 'The quor closes that 428 Represer have respon

In [60]:
embeded = embed(small_speeches)

In [61]:
embeded.shape

TensorShape([10000, 512])

In [62]:
query = embed(["What reforms were adopted by the 110th Congress?"])

In [34]:
query.numpy().shape

(1, 512)

In [63]:
scores = cosine_similarity(embeded.numpy(), query.numpy())
results = (np.flip(np.argsort(scores, axis=0)))

In [64]:
results.shape

(10000, 1)

In [65]:
for i in results[0:5, 0]:
    print(change_comma(sm_speeches[i]))
    print()

Congratulations. You are now Members of the 111th Congress.

Congratulations. You are now Members of the 111th Congress.

Congratulations. You are now a Member of the 111th Congress.

Madam Speaker, let me congratulate you for your reelection as Speaker of the House. It is an honor that you tave served with great distinction and verve, I look forward to more of your continued leadership in the 111th Congress. Mr. Speaker, I rise today in support of H. Res, 5. Adopting the rules for the One Hundred Eleventh Congress. The House Rules Package provides commonsense reforms that will enable Congress to work more efficiently for America. In the 110th Congress. Democrats put forth critical measures to restore integrity and accountability to the House. These reforms were the most sweeping ethics and lobbying reforms since Watergate and has changed the way Congress does business in Washington. The reforms adopted by the 110th Congress included banning gifts from lobbyists, prohibiting the use of

In [66]:
THRESHOLD = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:10, 0] ]
contexts = [ (small_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]

In [67]:
question_df = pd.DataFrame.from_records([ {
        'question': "What do Democrats believe about tax increases?",
        'context':  ctx
} for (ctx,s) in contexts ])
    
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)

In [68]:
answer_df

,score,start,end,answer
2,0.575306,0,16,Congratulations.
0,0.372675,0,16,Congratulations.
1,0.372675,0,16,Congratulations.
8,0.366115,263,274,opposed it.
3,0.302926,478,569,Democrats put forth critical measures to resto...
5,0.146672,501,575,Democratic majority is severely restricting op...
9,0.037741,430,471,Democratic Caucus from the Presidentelect
6,0.018020,1427,1481,every taxpayer should be up in arms over these...
4,0.005402,129,168,The bill was passed. as follows: S. 181
7,0.000319,1622,1654,I reserve the balance of my time.


In [69]:
query = embed(["What do you believe about tax increases?"])
scores = cosine_similarity(embeded.numpy(), query.numpy())
results = (np.flip(np.argsort(scores, axis=0)))
for i in results[0:5, 0]:
    print(change_comma(sm_speeches[i]))
    print()
THRESHOLD = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:10, 0] ]
contexts = [ (small_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]
question_df = pd.DataFrame.from_records([ {
        'question': "What do you believe about tax increases?",
        'context':  ctx
} for (ctx,s) in contexts ])
    
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)
answer_df

Thank you. Congressman AKIN. And you know, when you showed the chart over there about the revenue, the dip and then ultimately as taxes were cut. Federal revenues actually increased. The same thing happened under President Reagan when President Reagan cut taxes, I think one of the myths that is out there is that the deficit grew. Some people tried to attribute that to the tax cuts. But if you really go and look, youll see a similar chart, youll actually see an increase in revenue. Unfortunately, you had a Democraticcontrolled Congress that spent even more money than the new money that did come in. But in fact, more money came in as taxes were cut. And so I hope we use history as a guide. As you talked about earlier, there is no bill filed yet on this economic stimulus plan. We are expecting in the next week to possibly 2 weeks, nere will be a bill filed. And unfortunately, right now what youve got is a bidding war. What started off as maybe a $400 or $500 billion proposal has now reach

,score,start,end,answer
1,0.546953,449,485,keeps the current tax rate the same.
4,0.495226,1427,1483,increasing the average annual productivity of ...
2,0.458218,0,25,Very much I like tax cuts.
3,0.195624,240,298,the money wasnt coming back until we lowered t...
0,0.130088,146,182,Federal revenues actually increased.
7,0.076859,1373,1422,they can have an increased investment tax credit.
9,0.033735,613,655,people getting a tax break at every level.
6,0.020107,917,975,average taxpayers saw their earnings decline a...
5,0.005408,30,51,They were flat wrong.
8,0.000003,1061,1119,They cant do it because they dont have the mon...


In [42]:
query = embed(["How much working women make compared to men in the same position in the United States?"])
scores = cosine_similarity(embeded.numpy(), query.numpy())
results = (np.flip(np.argsort(scores, axis=0)))
THRESHOLD = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:10, 0] ]
contexts = [ (small_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]
question_df = pd.DataFrame.from_records([ {
        'question': "What do Democrats believe about tax increases?",
        'context':  ctx
} for (ctx,s) in contexts ])
    
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)
answer_df

,score,start,end,answer
4,6.225415e-02,845,897,Please support equal pay for equal work and vo...
7,1.866715e-02,87,142,women receive equal opportunity to be trained ...
1,1.599180e-02,452,465,family issue.
3,7.326970e-03,1110,1160,this statistic is unlikely to change anytime s...
5,3.693898e-04,1116,1157,families are forced to make do with less.
8,2.004129e-04,398,477,we wouldnt need legislation to reinforce a con...
2,3.591713e-06,1188,1245,closing the pay gap in Michigan and throughout...
0,3.383946e-06,1046,1073,growing economic recession.
9,1.623113e-06,678,728,women have reached full equality in the workpl...
6,6.636560e-07,1539,1544,1963.


In [43]:
contexts

[('Mr. Speaker. I rise today in support of two critical bills: the Lilly Ledbetter Fair Pay Act and the Paycheck Fairness Act. More than 40 years have passed since the passage of the Equal Pay Act. Unfortunately. women still earn less than men for the same work. and that is unfair. Women earn 78 cents for every dollar earned by men. The pay gap is even more severe amongst minority women. African American women earn 69 cents. Hispanic women earn just 56 cents on the dollar. The story of Lilly Ledbetter is a case in point of the barriers faced by women today. Its time for a change. Simply because of her gender. Ms. Ledbetter was paid 20 percent less for performing the same job that her male colleagues performed. Its unfair. The gap doesnt just affect women. It affects the Nations economy. Women represent 41 percent of the total heads of households and sole income earners in this country. We cannot afford to weaken the ability of our breadwinners to pay for the basics. like groceries. chi